## Updated Code for Implementing VAR Model

In [1]:
# load modules
import numpy as np
import pandas as pd
import pmdarima as pm
import statsmodels.api as sm
from sktime.transformations.series.difference import Differencer
from sktime.performance_metrics.forecasting import mean_absolute_error, MeanAbsoluteError
from sktime.forecasting.var import VAR
from data_protection_functions import *
from data_processing_functions import *
from forecasting_functions import *
# nice time series plots
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import ForecastingGridSearchCV, ExpandingWindowSplitter, SlidingWindowSplitter
from sktime.forecasting.compose import TransformedTargetForecaster

In [2]:
# import weekly finance time series
# ignore header and skip the first row to use integers as column names
full_data = pd.read_csv("../../Data/Train/Clean/full_m3_monthly_micro_clean.csv", header=None, skiprows=1)

In [3]:
# convert to a list of series, potentially with different lengths
full_data = [x.dropna() for _, x in full_data.iterrows()]

***

In [4]:
# forecast horizon
h = 1

In [5]:
Y = [x.iloc[:-h] for x in full_data]
Test = [x.iloc[-h:] for x in full_data]

## Step 2: Apply Data Protection to Generate Protected Series

In [6]:
Y_protected = apply_data_protection(Y, epsilon=1)

***

## Step 3: Pre-process the data.

In [7]:
Y_processed, Y_last_window, Y_last_window_trend, _, full_lags  = pre_process(ts_data=Y, 
                                                                  target_forecast_period=h,
                                                                  log=True, 
                                                                  make_stationary=True, 
                                                                  sp=12)

Y_protected_processed, Y_protected_last_window, Y_protected_last_window_trend, _, full_lags_protected = pre_process(ts_data=Y_protected, 
                                                                                               target_forecast_period=h,
                                                                                               log=True,  
                                                                                               make_stationary=True, 
                                                                                               sp=12)

***

In [8]:
# forecasts_original = post_process(full_ts_data=Y, 
#                                   forecasts=Y_processed, 
#                                   target_forecast_period=h,
#                                   full_lags=full_lags,
#                                   log=True,
#                                   make_stationary=True,
#                                   sp=12)

***

***

## Step 4: Train Models and Generate Forecasts

In [9]:
# param_grid = {"maxlags": [10, 15, 20, 25, 30, 35, 40], "trend": ["c", "ct", "ctt", "n"]}
# param_grid = {"maxlags": [10, 25, 40], "trend": ["c", "n"]}

In [10]:
forecasts_original = train_and_forecast(ts_data=Y_processed,
                                        horizon_length=h,
                                        forecasting_model="VAR")

forecasts_protected = train_and_forecast(ts_data=Y_protected_processed,
                                         horizon_length=h,
                                         forecasting_model="VAR")

***

## Post Process the Forecasts

In [11]:
forecasts_original = post_process(full_ts_data=Y, 
                                  forecasts=forecasts_original, 
                                  target_forecast_period=h,
                                  log=True,
                                  make_stationary=True,
                                  sp=12,
                                  full_lags=full_lags)

forecasts_protected = post_process(full_ts_data=Y_protected, 
                                   forecasts=forecasts_protected, 
                                   target_forecast_period=h,
                                   log=True,
                                   make_stationary=False,
                                   sp=12,
                                   full_lags=full_lags_protected)

In [12]:
forecasts_original

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
0,798.107469,3154.080207,4369.97244,6368.258517,8996.215254,328.838628,2804.273732,2649.00548,2617.154035,4247.259117,...,2403.334645,3822.946249,5070.454238,7166.416515,4839.675524,8002.019398,5174.22819,4054.280824,8233.050653,2685.604593


In [13]:
forecasts_protected

,0,1,2,3,4,5,6,7,8,9,...,464,465,466,467,468,469,470,471,472,473
0,224.496251,69.291683,669.906393,144.206388,495.973634,491.635122,83.945809,2428.250298,52.314552,403.605092,...,301.449377,11232.002376,565.822681,9973.00331,3325.556729,3151.765838,5245.194967,1853.321809,8138.153155,296.740819


In [14]:
Test = pd.DataFrame([x.reset_index(drop=True) for x in Test]).T

In [15]:
mean_absolute_error(Test, forecasts_original)

753.1416838746653

In [16]:
mean_absolute_error(Test, forecasts_protected)

2839.318499290283

***